In [1]:
import numpy as np
import scipy.optimize 

In [2]:
def cole_turb(e_d):
    return (-2*np.log10(e_d/3.7))**-2

def coleb(Re,e_d):
    eqf = lambda f : f**-0.5+2*np.log10(e_d/3.7+2.51/Re/f**0.5)
    ffact = scipy.optimize.newton(eqf,cole_turb(e_d))
    return ffact

# factores conversion unidades
ft2m = 0.3048    # ft -> m
in2m = 0.0254    # in -> m
psi2pa = 6894.76 # psi -> Pa

# Diametro interno (mm) a partir del diametro nominal (in)
ced_40={'0'     : 0 ,
        '1/8'   : 6.8   ,
        '1/4'   : 9.2   ,
        '3/8'   : 12.5  ,
        '1/2'   : 15.8  ,
        '3/4'   : 20.9  ,
        '1'     : 26.6  ,
        '1 1/4' : 35.1  ,
        '1 1/2' : 40.9  ,
        '2'     : 52.5  ,
        '2 1/2' : 62.7  ,
        '3'     : 77.9  ,
        '3 1/2' : 90.1  ,
        '4'     : 102.3 ,
        '5'     : 128.2 ,
        '6'     : 154.1 ,
        '8'     : 202.7 ,
        '10'    : 254.5 ,
        '12'    : 303.2 ,
        '14'    : 333.4 ,
        '16'    : 381.0 ,
        '18'    : 428.7 ,
        '20'    : 477.9 ,
        '24'    : 574.7        
}

In [3]:
rho = 10**3 # kg/m3
mu = 10**-4 # Pa.s
g = 9.81

Di = np.zeros(12)
Di[0]=16
Di[1]=16
Di[2]=18
Di[3]=12
Di[4]=16
Di[5]=16
Di[6]=12
Di[7]=14
Di[8]=12
Di[9]=8
Di[10]=12
Di[11]=8

for i in range(len(Di)):
    Di[i]=ced_40[str(int(Di[i]))]*10**-3
At = np.multiply(np.pi/4.0,np.power(Di,2))

Li = np.zeros(12)
Li[0] = 1500
Li[1] = 1500
Li[2] = 2000
Li[3] = 2000
Li[4] = 2000
Li[5] = 1500
Li[6] = 1500
Li[7] = 4000
Li[8] = 4000
Li[9] = 4000
Li[10] = 1500
Li[11] = 1500

Li = np.multiply(Li,ft2m)

e = 4.6*10**-5 #m
e_d = np.divide(e,Di)

QA = 15.5*ft2m**3
QC = 1.5*ft2m**3
QE = 1*ft2m**3
QF = 4*ft2m**3
QG = 3*ft2m**3
QH = 3*ft2m**3
QI = 3*ft2m**3

def eo(v):
    # q corresponde al vector que contiene todos los flujos volumetricos
    q = np.multiply(v,At)
    eo1 = QA-q[0]-q[2] #A
    eo2 = q[0]-q[3]-q[1] #B
    eo3 = q[1]-QC-q[4] #C
    eo4 = q[2]-q[5]- q[7] #D
    eo5 = q[5]+q[3]-QE-q[8]-q[6] #E
    eo6 = q[6]+q[4]-QF-q[9] #F
    eo7 = q[7]-QG-q[10] #G
    eo8 = q[10]+q[8]-QH-q[11] #H
    eo9 = q[11]+q[9]-QI #I
    
    Re = np.multiply(rho/mu,np.multiply(v,Di))
    ffact = np.zeros(12)
    for i in range(12):
        ffact[i]=coleb(Re[i],e_d[i])
    hl = np.multiply(np.multiply(ffact,np.divide(Li,Di)),np.divide(np.power(v,2),2*g))
    
    eo10 = hl[0]+hl[3]-hl[2]-hl[5]
    eo11 = hl[0]+hl[1]+hl[4]-(hl[2]+hl[5]+hl[6])
    eo12 = hl[0]+hl[1]+hl[4]+hl[9]-(hl[2]+hl[7]+hl[10]+hl[11])
    return([eo1,eo2,eo3,eo4,eo5,eo6,eo7,eo8,eo9,eo10,eo11,eo12])


q_p = scipy.optimize.fsolve(eo,1*np.ones(12))
print(q_p)

[1.70281193 1.18369537 1.69578223 0.81970284 0.81113529 0.86898762
 0.91069333 1.66895838 0.88898066 1.39334941 0.84141645 1.23915292]


In [4]:
def eo2(v):
    # q corresponde al vector que contiene todos los flujos volumetricos
    q = np.multiply(v,At)
    eo1 = QA-q[0]-q[2] #A
    eo2 = q[0]-q[3]-q[1] #B
    eo3 = q[1]-QC-q[4] #C
    eo4 = q[2]-q[5]- q[7] #D
    eo5 = q[5]+q[3]-QE-q[8]-q[6] #E
    eo6 = q[6]+q[4]-QF-q[9] #F
    eo7 = q[7]-QG-q[10] #G
    eo8 = q[10]+q[8]-QH-q[11] #H
    eo9 = -q[11]+q[9]-QI #I
    
    Re = np.multiply(rho/mu,np.multiply(v,Di))
    ffact = np.zeros(12)
    for i in range(12):
        ffact[i]=coleb(Re[i],e_d[i])
    hl = np.multiply(np.multiply(ffact,np.divide(Li,Di)),np.divide(np.power(v,2),2*g))
    
    eo10 = hl[0]+hl[3]-hl[2]-hl[5]
    eo11 = hl[0]+hl[1]+hl[4]-(hl[2]+hl[5]+hl[6])
    eo12 = hl[0]+hl[1]+hl[4]+hl[9]+hl[11] - (hl[2]+hl[7]+hl[10])
    return([eo1,eo2,eo3,eo4,eo5,eo6,eo7,eo8,eo9,eo10,eo11,eo12])


q_p2 = scipy.optimize.fsolve(eo2,1*np.ones(12))

C:\Users\enriq\anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log10
  """


RuntimeError: Failed to converge after 50 iterations, value is nan.